In [ ]:
import keras
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Concatenate,Activation
from keras.layers import concatenate
from keras.layers import Conv2D, Conv2DTranspose, ZeroPadding2D,MaxPooling2D, Cropping2D, BatchNormalization
from keras.models import Model
from keras import metrics
from keras import backend as K
from keras import optimizers
from keras import losses
from keras.utils import plot_model
from keras.callbacks import Callback,ModelCheckpoint, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.losses import categorical_crossentropy
from keras.optimizers import Adam


import tensorflow as tf
import random
import time
import numpy as np
import skimage.io as io
import matplotlib
import matplotlib.pyplot as plt

import datetime

import cv2
%matplotlib inline
#import pydot
from PIL import Image
import glob
import os
#from pyemd import emd, emd_samples

In [ ]:
# Parameters initialization
nb_rows = 224   # X dimension of the image
nb_cols = 224   # Y dimesnion of the image
#total_frames = 30

nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

BS = 10
batch_size = BS

In [ ]:
from skimage.transform import rescale
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range = 0,
    width_shift_range = 0,
    height_shift_range = 0,
    vertical_flip = False,)


# batch_labels is the one hot representation of the output
def initialize_batch_data(batch_size):
    batch_data = np.zeros((batch_size, nb_rows, nb_cols, nb_channel)) 
    batch_labels = np.zeros((batch_size,nb_rows, nb_cols,1)) 
    return batch_data, batch_labels

# Image Croping
# def crop(image):
#     if image.shape[0] != image.shape[1]:
#         return image[0:120,20:140]
#     else:
#         return image

# Resizing the image based on dimension
def resize(image):
    return  cv2.resize(image, (nb_rows,nb_cols), interpolation = cv2.INTER_AREA)


def preprocess_img(img, mode):
    img = (img - img.min())/(img.max() - img.min())
    if mode == 'train':
        if np.random.randn() > 0:
            img = datagen.random_transform(img)
    return img

In [ ]:
def getData(mode,source_path):
    image_list=[]
    for filename in glob.glob(str(source_path)+'images/*'):
        path1 = os.path.basename(filename)
        image_list.append(path1)
#     paths = image_list[0:100]
    return image_list

In [ ]:
#Enahancing the sal_map images/ manual thresholding

def getBatchImage(folder_list,indexes, mode,source_path):
    list_IDs_temp = [folder_list[k] for k in indexes]
#     print("indexes",list_IDs_temp)
    X=[]
    Y=[]
    for index, h in enumerate(list_IDs_temp):
        I = io.imread(str(source_path)+"images/{}".format(h))
        resized = resize(I)
        new_image = preprocess_img(resized, mode)
        X.append(new_image)
        bin_label = np.zeros((224,224))
        label = io.imread(str(source_path)+"salMap/{}".format(h))
        resized = resize(label)
        labels = preprocess_img(resized, mode)
        for i in range(0,224):
            for j in range(0,224):
                if labels[i][j]<26:
                    bin_label[i][j] = 0
                elif labels[i][j]<51:
                    bin_label[i][j] = 0.111
                elif labels[i][j]<76:
                    bin_label[i][j] = 0.222
                elif labels[i][j]<102:
                    bin_label[i][j] = 0.333
                elif labels[i][j]<128:
                    bin_label[i][j] = 0.444
                elif labels[i][j]<154:
                    bin_label[i][j] = 0.556
                elif labels[i][j]<180:
                    bin_label[i][j] = 0.667
                elif labels[i][j]<206:
                    bin_label[i][j] = 0.778
                elif labels[i][j]<230:
                    bin_label[i][j] = 0.889                  
                else:
                    bin_label[i][j] = 1
        Y.append(bin_label)
    X = np.array(X)
    Y = np.array(Y)
    Y=Y[:,:,:,np.newaxis]
    return X,[Y,Y,Y,Y]

def get_newBatch(folder_list,batch_size,index):
    indexes = np.arange(len(folder_list))
    np.random.shuffle(indexes)
    indexes = indexes[index*batch_size:(index+1)*batch_size]
    return indexes

In [ ]:
#generator function
def generator(batch_size,source_path,mode='train'):
    
    while True:
        folder_list = getData(mode,source_path)

        num_batches = len(folder_list)//batch_size # calculate the number of batches
        print('No of Batch : ', num_batches,' mode: ' , mode)
        for batch in range(num_batches): # we iterate over the number of batches
            # you yield the batch_data and the batch_labels, remember what does yield do
            indexes = get_newBatch(folder_list,batch_size,batch)
            yield getBatchImage(folder_list,indexes,mode,source_path)

        # Code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            indexes = get_newBatch(folder_list,batch_size,batch)
            yield getBatchImage(folder_list,indexes,mode,source_path)

In [ ]:
# BS=20
# batch_size = BS
training_Path = "/Users/s0c02nj/Downloads/lagdata/train/"
validation_Path = "/Users/s0c02nj/Downloads/lagdata/validation/"
# train_generator = generator(batch_size, training_Path, mode='train')
# validation_generator = generator(batch_size, validation_Path, mode='train')

In [ ]:
# for t in validation_generator:
#     print(t[0].shape,t[1][0].shape)

In [ ]:
curr_dt_time = datetime.datetime.now()

def get_step_per_epoch(num_train_sequences, b_size):
    if (num_train_sequences%b_size) == 0:
        steps_per_epoch = int(num_train_sequences/b_size)
    else:
        steps_per_epoch = (num_train_sequences//b_size) + 1
    return steps_per_epoch

def get_validation_steps(num_val_sequences, b_size):
    if (num_val_sequences%b_size) == 0:
        validation_steps = int(num_val_sequences/b_size)
    else:
        validation_steps = (num_val_sequences//b_size) + 1
    return validation_steps
    

def callbacks_list(model_name, factor_rate, epoch_patience):
    print('factor_rate: ',factor_rate)
    model_name = model_name + '_'+'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    if not os.path.exists(model_name):
        os.mkdir(model_name)
    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{bafc_accuracy:.5f}-{val_loss:.5f}-{val_bafc_accuracy:.5f}.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_bafc_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    LR = ReduceLROnPlateau(monitor='val_bafc_loss', factor=factor_rate, patience=epoch_patience, cooldown=1, verbose=1, mode='auto', min_delta=0.0001) # write the REducelronplateau code here
    callbacks_list = [checkpoint, LR]   
    return callbacks_list



In [ ]:
class AttentionModel:
  def __init__(self):
        self.batch_size = BS
        self.epochs = 5 #10
        self.lr = 0.001

        
  #Defines the model architecture      
  def DeepAttentionModel(self):
    img_rows, img_cols, img_chns = 224, 224, 3
    if K.image_data_format() == 'channels_first':
        original_img_size = (img_chns, img_rows, img_cols)
    else:
        original_img_size = (img_rows, img_cols, img_chns)
    
    #print("###########################################")
    #print("original_img_size ", original_img_size)
    #print("###########################################")
    
    self.x = Input(shape=original_img_size)
    padded_x = ZeroPadding2D(padding=(35), data_format="channels_last")(self.x)
    
    
    #'Encoder'
    conv_1_1 = Conv2D(64, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_x)
    padded_conv_1_1 = ZeroPadding2D(padding=(1), data_format="channels_last")(conv_1_1)
    conv_1_2 = Conv2D(64, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_conv_1_1)
    pool_1 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format="channels_last")(conv_1_2)
    padded_input_pool_1=  ZeroPadding2D(padding=(1), data_format="channels_last")(pool_1)
    
    conv_2_1 = Conv2D(128, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_input_pool_1)
    padded_input_conv_2_1 = ZeroPadding2D(padding=(1), data_format="channels_last")(conv_2_1)
    conv_2_2 = Conv2D(128, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_input_conv_2_1)
    pool_2 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format="channels_last")(conv_2_2)
    padded_input_pool_2=  ZeroPadding2D(padding=(1), data_format=None)(pool_2)
    
    conv_3_1 = Conv2D(256, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_pool_2)
    padded_input_conv_3_1 = ZeroPadding2D(padding=(1), data_format=None)(conv_3_1)
    conv_3_2 = Conv2D(256, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_3_1)
    padded_input_conv_3_2 = ZeroPadding2D(padding=(1), data_format=None)(conv_3_2)
    conv_3_3 = Conv2D(256, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_3_2)
    pool_3 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(conv_3_3)
    padded_input_pool_3 =  ZeroPadding2D(padding=(1), data_format=None)(pool_3)
    
    conv_4_1 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_pool_3)
    padded_input_conv_4_1 = ZeroPadding2D(padding=(1), data_format=None)(conv_4_1)
    conv_4_2 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_4_1)
    padded_input_conv_4_2 = ZeroPadding2D(padding=(1), data_format=None)(conv_4_2)
    conv_4_3 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_4_2)
    pool_4 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding='same', data_format=None)(conv_4_3)
    padded_input_pool_4 =  ZeroPadding2D(padding=(1), data_format=None)(pool_4)
    
    conv_5_1 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_pool_4)
    padded_input_conv_5_1 = ZeroPadding2D(padding=(1), data_format=None)(conv_5_1)
    conv_5_2 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_5_1)
    padded_input_conv_5_2 = ZeroPadding2D(padding=(1), data_format=None)(conv_5_2)
    conv_5_3 = Conv2D(512, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(padded_input_conv_5_2)
    
    
    #'Decoder'
    deconv_5_1 = Conv2DTranspose(512,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(conv_5_3)
    deconv_5_2 = Conv2DTranspose(256,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_5_1)
    deconv_5_3 = Conv2DTranspose(128,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_5_2)
    deconv_5_4 = Conv2DTranspose(64,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_5_3)
    
    attention1 = Conv2D(1, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(deconv_5_4 )
    attention1c = Cropping2D(cropping= ((54,54),(54,54)), data_format=None)(attention1)
    self.bn_attention1c = BatchNormalization(name='ba1c')(attention1c)
    
    deconv_4_1 = Conv2DTranspose(256,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(conv_4_3)
    deconv_4_2 = Conv2DTranspose(128,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_4_1)
    deconv_4_3 = Conv2DTranspose(64,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_4_2)
    
    attention2 = Conv2D(1, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(deconv_4_3 )
    attention2c = Cropping2D(cropping= ((42,42),(42,42)))(attention2)
    self.bn_attention2c = BatchNormalization(name='ba2c')(attention2c)
    
    deconv_3_1 = Conv2DTranspose(128,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(conv_3_3)
    deconv_3_2 = Conv2DTranspose(64,kernel_size=(4, 4),strides=(2, 2), padding='valid', activation='relu')(deconv_3_1)
    
    attention3 = Conv2D(1, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu')(deconv_3_2)
    attention3c = Cropping2D(cropping= ((36,36),(36,36)))(attention3)
    self.bn_attention3c = BatchNormalization(name='ba3c')(attention3c)
    
    attention = concatenate([attention1c,attention2c,attention3c])
    padded_attention = ZeroPadding2D(padding=(1))(attention)
    final_attention =  Conv2D(1, kernel_size=(3, 3),strides=(1, 1), padding='valid', activation='relu',data_format="channels_last")(padded_attention)
    self.bn_final_attention = BatchNormalization(name='bafc')(final_attention)
    
    
    self.model = Model(inputs=self.x, outputs=[self.bn_attention1c,self.bn_attention2c,self.bn_attention3c,self.bn_final_attention ])
    
    
    def custom_loss(y_true, y_pred):
        
        loss1 = losses.binary_crossentropy(y_true,self.bn_attention1c)
        loss2 = losses.binary_crossentropy(y_true,self.bn_attention2c)
        loss3 = losses.binary_crossentropy(y_true,self.bn_attention3c)
        loss4 = losses.binary_crossentropy(y_true,self.bn_final_attention)

        return (loss1+loss2+loss3+loss4)/4.0

    sgd = optimizers.SGD(lr=self.lr) #Stochastic Gradient Descent Optimizer
    self.loss = custom_loss
    self.model.compile(optimizer = sgd , loss = self.loss, metrics=['accuracy'])


  def Train(self):
    global output
    global input_image_full
    global req_output_image_full
    factor=0.20
    patience=5
    model_name_folder = 'conv2d_attention'
    
    num_train_sequences = len(getData('train',training_Path))
    num_val_sequences = len(getData('val',validation_Path))
    
    steps_per_epoch = get_step_per_epoch(num_train_sequences, batch_size)
    validation_steps = get_validation_steps(num_val_sequences, batch_size)
    
    train_generator = generator(batch_size, training_Path, mode='train')
    validation_generator = generator(batch_size, validation_Path, mode='val')
    
    callbacks_model = callbacks_list(model_name_folder,factor,patience)
    
#     X_train,Y_train = training_generator[0]
#     X_test,Y_test = validation_generator[0]
    # History object stores loss and accuracy
    hist_obj = self.model.fit_generator(train_generator,validation_data=validation_generator,
                                        steps_per_epoch=steps_per_epoch,validation_steps=validation_steps,
                                        epochs=self.epochs,verbose=1,callbacks=callbacks_model,initial_epoch=0)
    
#     ,callbacks=callbacks_list_for_Conv2D_plus_lstm_model
#     output = self.model.predict(X_test)
#     output = np.array(output)
    
    # Making data global, to be used for visualization
#     input_image_full = X_train[:,:,:]
#     req_output_image_full =Y_train[0][:,:,:]
    
    return hist_obj
 
  
#   # Prints the model architecture
  def get_Model_Summary(self):      
      print(self.model.summary())
    

#   def Visualise_Output(self,idx):
#     self.input_image =input_image_full[idx,:,:]
#     self.req_output_image =req_output_image_full[idx,:,:]
#     pred_image_1 = output[0,idx,:,:,:]
#     pred_image_2 = output[1,idx,:,:,:]
#     pred_image_3 = output[2,idx,:,:,:]
#     pred_image_4 = output[3,idx,:,:,:]
#     fig = plt.figure(figsize=(20,10))
#     plt.subplot(231)
#     plt.imshow((self.input_image.reshape(224,224,3)), interpolation='none')
#     plt.title("Input - ")
#     ax = plt.gca()
#     ax.grid(False)
#     plt.subplot(232)
#     plt.imshow(self.req_output_image.reshape(224,224)*255,interpolation='none')
#     plt.title("Ground Truth Attention")
#     ax = plt.gca()
#     ax.grid(False)
#     plt.subplot(233)
#     plt.imshow(pred_image_1.reshape(224,224)*255,interpolation='none')
#     plt.title("Predicted Attention 1 ")
#     ax = plt.gca()
#     ax.grid(False)
#     plt.subplot(234)
#     plt.imshow(pred_image_2.reshape(224,224)*255,interpolation='none')
#     plt.title("Predicted Attention 2 ")
#     ax = plt.gca()
#     ax.grid(False)
#     plt.subplot(235)
#     plt.imshow(pred_image_3.reshape(224,224)*255,interpolation='none')
#     plt.title("Predicted Attention 3 ")
#     ax = plt.gca()
#     ax.grid(False)
#     plt.subplot(236)
#     plt.imshow(pred_image_4.reshape(224,224)*255,interpolation='none')
#     plt.title("Predicted Final Attention ")
#     ax = plt.gca()
#     ax.grid(False)
#     plot_name = 'plot' + '_'+str(idx)+'.png'
#     print(plot_name)
#     fig.savefig(plot_name)
    
    

In [ ]:
deconvNet = AttentionModel()
deconvNet.DeepAttentionModel()

In [ ]:
deconvNet.get_Model_Summary()

In [ ]:
hist_obj = deconvNet.Train()